In [35]:
from elasticsearch import Elasticsearch
import pickle

In [38]:
es = Elasticsearch("http://localhost:9200")
resp = es.info()
resp

ObjectApiResponse({'name': '163dd7feee2c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'qiPQpuGBSwGK5RW2XA7YHw', 'version': {'number': '8.8.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '98e1271edf932a480e4262a471281f1ee295ce6b', 'build_date': '2023-06-26T05:16:16.196344851Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [39]:
index_name = 'products'

In [40]:
file_name = '.\Data\df_dict.pkl'
with open(file_name, "rb") as file:
    df_dict = pickle.load(file)

## Store the data to the elasticsearch

In [45]:
# for i, product in enumerate(df_dict):
#     es.index(index=index_name, id = product['Product_id'], body=product)

## Define your count query to get the total number of documents in the index.

In [42]:
count_query = {
    "query": {
        "match_all": {}
    }
}

# Perform the count query
response = es.count(index=index_name, body=count_query)

# Extract the total count of documents
total_documents = response['count']

print(f"Total number of documents in the index '{index_name}': {total_documents}")

<ipython-input-42-c25c9c64f132>:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.count(index=index_name, body=count_query)


Total number of documents in the index 'products': 488


In [43]:
def recommend_products(keywords, num_results=5):
    res = es.search(
        index="products",
        body={
            "query": {
                "multi_match": {
                    "query": keywords,
                    "fields": ['Product_Name', 'Product_Name_en', 'Description', 'Description_en'],
                    "type": "cross_fields",
                    "operator": "and"
                }
            },
            "size": num_results
        }
    )

    recommendations = []
    for hit in res['hits']['hits']:
        recommendations.append(hit['_source'])

    return recommendations

In [44]:
recommend_products('葡萄')

<ipython-input-43-f788714ce77d>:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(


[{'Product_id': 'CU0009108106',
  'Product_Name': '福氣永豐(葡萄酒)',
  'Product_Name_en': 'Viachi Spirit – Abundance (Grape)',
  'Description': '天癒巨峰葡萄醇釀，濃烈豐厚，温潤純粹，值得品味珍藏。 \n \n 無添加人工香料及食用酒精\n \n (58%)',
  'Description_en': 'Made from healthy grapes cultivated using the AgriDragon biomimicry farming technique, this unique spirit is rich and well-balanced with layers of flavors. No artificial flavorings or edible alcohol is added. (58%)'},
 {'Product_id': 'CU0007812108',
  'Product_Name': '仰照式紅色光譜植物燈',
  'Product_Name_en': 'shine upwards type red spectrum plant light',
  'Description': '⚫ 提高著果率，適用於葡萄、百香果及瓜果類植物\n \n ⚫ 促進植物生長，適用於各式農作物。⚫ 625nm/660nm雙紅光譜設計，包含一般紅光與植物專用紅光，具有更佳吸收效率。 \n \n ⚫ 專為 LED 芯片「定電流」設計，低功率損耗，降低工作溫度 \n \n ⚫ 高透光壓克力圓罩，採用圓罩設計光線更均勻分布。 \n \n ⚫ 鋁基板設計，提高散熱效果，使 LED 芯片更耐用，延長使用週期降低光衰。',
  'Description_en': '⚫ Improve the fruit setting rate, suitable for grapes, passion fruit and melon and fruit plants\n \n ⚫ Promote plant growth, suitable for all kinds of crops. \n \n ⚫ 625nm/660nm dual-r

In [18]:
# # Send the DELETE request to delete the entire index
# response = es.indices.delete(index=index_name)

# # Check if the operation was successful
# if response["acknowledged"]:
#     print(f"The index '{index_name}' has been deleted.")
# else:
#     print(f"Failed to delete the index '{index_name}'.")

The index 'products' has been deleted.
